In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow import keras
import tensorflow as tf

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler,ReduceLROnPlateau

from tensorflow.keras import layers
from tensorflow.keras.applications import MobileNetV2,EfficientNetB0,EfficientNetB4

import warnings
warnings.filterwarnings("ignore")

In [8]:
img_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1.0/255.0,
    zoom_range=0.1,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)

In [12]:
train_ds = img_datagen.flow_from_directory('dataset/images/train/', class_mode = 'categorical', batch_size=32, target_size=(48,48), color_mode='grayscale')
val_ds = img_datagen.flow_from_directory('dataset/images/validation/', class_mode = 'categorical', batch_size=32, target_size=(48,48), color_mode='grayscale')
test_ds = img_datagen.flow_from_directory('dataset/images/test/', class_mode = 'categorical', batch_size=32, target_size=(48,48), color_mode='grayscale')

Found 26921 images belonging to 7 classes.
Found 7066 images belonging to 7 classes.
Found 1900 images belonging to 7 classes.


In [13]:
for image_batch, labels_batch in train_ds:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

(32, 48, 48, 1)
(32, 7)


### Simple CNN

In [5]:
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),

    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),

    layers.Dense(7, activation='softmax')
])

In [6]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
save_best = tf.keras.callbacks.ModelCheckpoint("Model.h5",monitor='val_accuracy',save_best_only=True, verbose=1)

In [8]:
model.fit(train_ds, validation_data=val_ds, epochs=100, callbacks=[save_best], shuffle=True)

Epoch 1/100
421/421 [==============================] - ETA: 0s - loss: 2.7368 - accuracy: 0.1942
Epoch 1: val_accuracy improved from -inf to 0.25177, saving model to Model.h5
421/421 [==============================] - 128s 296ms/step - loss: 2.7368 - accuracy: 0.1942 - val_loss: 1.9295 - val_accuracy: 0.2518
Epoch 2/100
421/421 [==============================] - ETA: 0s - loss: 2.3871 - accuracy: 0.2280
Epoch 2: val_accuracy improved from 0.25177 to 0.32522, saving model to Model.h5
421/421 [==============================] - 22s 51ms/step - loss: 2.3871 - accuracy: 0.2280 - val_loss: 1.7564 - val_accuracy: 0.3252
Epoch 3/100
421/421 [==============================] - ETA: 0s - loss: 2.1828 - accuracy: 0.2579
Epoch 3: val_accuracy improved from 0.32522 to 0.34376, saving model to Model.h5
421/421 [==============================] - 22s 52ms/step - loss: 2.1828 - accuracy: 0.2579 - val_loss: 1.6833 - val_accuracy: 0.3438
Epoch 4/100
421/421 [==============================] - ETA: 0s - los

---

### Model Test Metric

In [23]:
model = tf.keras.models.load_model('Model.h5')

In [25]:
loss, acc = model.evaluate(test_ds, verbose=1)

30/30 [==============================] - 12s 393ms/step - loss: 1.1957 - accuracy: 0.5558


---
---

In [17]:
import kerastuner as kt

def build_model(hp):
    model = tf.keras.Sequential()
    
    model.add(layers.Conv2D(
        filters=hp.Int('filters_1', min_value=32, max_value=64, step=32),
        kernel_size=(3, 3),
        activation='relu',
        input_shape=(48, 48, 1)
    ))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(hp.Float('dropout_1', min_value=0.2, max_value=0.5, step=0.1)))
    
    model.add(layers.Conv2D(
        filters=hp.Int('filters_2', min_value=64, max_value=128, step=64),
        kernel_size=(3, 3),
        activation='relu'
    ))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(hp.Float('dropout_2', min_value=0.2, max_value=0.5, step=0.1)))
    
    model.add(layers.Conv2D(
        filters=hp.Int('filters_3', min_value=128, max_value=256, step=128),
        kernel_size=(3, 3),
        activation='relu'
    ))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(hp.Float('dropout_3', min_value=0.2, max_value=0.5, step=0.1)))
    
    model.add(layers.Flatten())
    model.add(layers.Dense(
        units=hp.Int('dense_units', min_value=128, max_value=512, step=128),
        activation='relu'
    ))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(hp.Float('dense_dropout', min_value=0.3, max_value=0.6, step=0.1)))
    
    model.add(layers.Dense(7, activation='softmax'))
    
    model.compile(
        optimizer=hp.Choice('optimizer', ['adam', 'sgd']),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

In [18]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=3,
    directory='my_dir',
    project_name='hyperparameter_tuning'
)

Reloading Tuner from my_dir\hyperparameter_tuning\tuner0.json


In [19]:
tuner.search(train_ds, validation_data=val_ds, epochs=10)

best_model = tuner.get_best_models(num_models=1)[0]

best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]
print(best_hyperparameters.values)

Trial 7 Complete [00h 09m 18s]
val_accuracy: 0.5355222225189209

Best val_accuracy So Far: 0.5355222225189209
Total elapsed time: 00h 57m 10s

Search: Running Trial #8

Value             |Best Value So Far |Hyperparameter
32                |64                |filters_1
0.4               |0.4               |dropout_1
64                |64                |filters_2
0.4               |0.2               |dropout_2
128               |256               |filters_3
0.2               |0.2               |dropout_3
128               |256               |dense_units
0.5               |0.4               |dense_dropout
sgd               |adam              |optimizer

Epoch 1/10
842/842 [==============================] - 20s 23ms/step - loss: 2.1451 - accuracy: 0.2173 - val_loss: 1.9467 - val_accuracy: 0.2523
Epoch 2/10
842/842 [==============================] - 18s 22ms/step - loss: 1.8029 - accuracy: 0.2660 - val_loss: 1.9474 - val_accuracy: 0.2593
Epoch 3/10
842/842 [==============================]

KeyboardInterrupt: 